In [ ]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.3/934.3 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 74.5 MB/s eta 0:00:00


In [ ]:
#!/usr/bin/env python3
# ----------------------------
# Baseline model for binarized MNIST usind a simple fully-connected (dense) neural network
# ----------------------------
import tensorflow as tf
from tensorflow.keras import layers, models
from pennylane.data import load as qml_load

/usr/local/lib/python3.12/dist-packages/pennylane/__init__.py:209: RuntimeWarning: PennyLane is not yet compatible with JAX versions > 0.6.2. You have version 0.7.2 installed. Please downgrade JAX to 0.6.2 to avoid runtime errors using python -m pip install jax~=0.6.0 jaxlib~=0.6.0
  warnings.warn(


In [ ]:
# ----------------------------
# Load binarized MNIST
# ----------------------------
def load_binarized_mnist():
    [ds] = qml_load("other", name="binarized-mnist")

    x_train = ds.train["inputs"].astype("float32")
    y_train = ds.train["labels"]

    x_test = ds.test["inputs"].astype("float32")
    y_test = ds.test["labels"]

    return (x_train, y_train), (x_test, y_test)

In [ ]:
# ----------------------------
# Baseline MLP
# ----------------------------
"""
Input layer of shape (784,).
First hidden layer.
Second hideen layer.
Output layer with 10-class classification.
"""
def build_baseline_model(input_dim=784, num_classes=10):
    model = models.Sequential(
        [
            layers.Input(shape=(input_dim,)),
            layers.Dense(128, activation="relu"),
            layers.Dense(64, activation="relu"),
            layers.Dense(num_classes, activation="softmax"),
        ]
    )
    return model

In [ ]:
# ----------------------------
# Main
# ----------------------------
def main():
    (x_train, y_train), (x_test, y_test) = load_binarized_mnist()

    model = build_baseline_model()
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )

    model.summary()

    # Early stopping on validation loss
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss", patience=3, restore_best_weights=True
        )
    ]

    history = model.fit(
        x_train,
        y_train,
        epochs=20,
        batch_size=128,
        validation_split=0.1,
        callbacks=callbacks,
        verbose=2,
    )

    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
    print(f"Baseline dense model – Test accuracy: {test_acc:.4f}, loss: {test_loss:.4f}")

In [ ]:
if __name__ == "__main__":
    main()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 109,386 (427.29 KB)

 Trainable params: 109,386 (427.29 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
352/352 - 5s - 15ms/step - accuracy: 0.8859 - loss: 0.3989 - val_accuracy: 0.9306 - val_loss: 0.2279
Epoch 2/20
352/352 - 2s - 6ms/step - accuracy: 0.9522 - loss: 0.1627 - val_accuracy: 0.9402 - val_loss: 0.1942
Epoch 3/20
352/352 - 2s - 7ms/step - accuracy: 0.9671 - loss: 0.1116 - val_accuracy: 0.9462 - val_loss: 0.1601
Epoch 4/20
352/352 - 2s - 6ms/step - accuracy: 0.9759 - loss: 0.0820 - val_accuracy: 0.9546 - val_loss: 0.1540
Epoch 5/20
352/352 - 3s - 9ms/step - accuracy: 0.9821 - loss: 0.0625 - val_accuracy: 0.9574 - val_loss: 0.1389
Epoch 6/20
352/352 - 2s - 7ms/step - accuracy: 0.9862 - loss: 0.0462 - val_accuracy: 0.9592 - val_loss: 0.1338
Epoch 7/20
352/352 - 2s - 6ms/step - accuracy: 0.9898 - loss: 0.0351 - val_accuracy: 0.9554 - val_loss: 0.1484
Epoch 8/20
352/352 - 2s - 6ms/step - accuracy: 0.9926 - loss: 0.0271 - val_accuracy: 0.9584 - val_loss: 0.1496
Epoch 9/20
352/352 - 2s - 6ms/step - accuracy: 0.9948 - loss: 0.0203 - val_accuracy: 0.9616 - val_loss: 0.1426
